# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

In [2]:
def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики доля правильных ответов (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = load_iris()
X = pd.DataFrame(data=data.data, columns=data.feature_names)
y = pd.Series(data.target)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [4]:
# класс ноды-решения, хранит в себе порог фичи и её индекс, ссылается на двух ребенков
class TreeNodeDecision():
  def __init__(self, feature, threshold, left, right):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right

In [5]:
# класс ноды-класса, хранит в себе предполагаемый класс объекта
class TreeNodeClass():
  def __init__(self, _class):
    self._class = _class

In [6]:
class MyDecisionTreeClassifier():
  def __init__(self, min_samples_leaf=5, max_depth=5, purity=True):
    self.root = None
    self.min_samples_leaf = min_samples_leaf
    self.max_depth = max_depth
    self.purity = purity

  # функция для отрисовки ноды на заданной глубине
  def draw_node(self, node, depth, special_symbol=""):
    depth_symbol = "|   "
    node_symbol = "|--- "
    s = "\n"
    for d in range(depth - 1):
      s += depth_symbol
    s += node_symbol
    if type(node) is TreeNodeClass:
      s += "class: " + str(node._class)
    else:
      s += "feature_" + str(node.feature) + special_symbol + str(node.threshold)
    return s

  # функция для отрисовки всего дерева с заданной ноды
  def draw_tree(self, node, depth=0):
    s = ""
    depth += 1
    if type(node) is TreeNodeDecision:
      s += self.draw_node(node, depth, " <= ") + self.draw_tree(node.left, depth) 
      s += self.draw_node(node, depth, " > ") + self.draw_tree(node.right, depth)
    elif type(node) is TreeNodeClass:
      s += self.draw_node(node, depth)
    return s

  # функция для предсказания класса объекта. циклично спускается по дереву, пока не встретит первый листок (ноду-класс)
  def predict(self, x):
    predictions = []
    for index, row in x.iterrows():
      current_node = self.root
      while (type(current_node) is not TreeNodeClass):
        current_node = current_node.left if (row[current_node.feature] <= current_node.threshold) else current_node.right
      predictions.append(current_node._class)
    return predictions

  # функция для проверки критерия остановки (нода -> листок)
  # проверка на purity и наличие в ноде менее, чем двух минимальных "размеров" листков
  # т.к. нода-решение должна иметь возможность разбиться на 2 листка -> содержать в себее более, чем два минимальных "размера" листков
  def node_stop_check(self, x):
    return len(x) < self.min_samples_leaf * 2 or (len(x.unique()) == 1 and self.purity)
    
  # функция для разделения дерева
  def tree_split(self, x, feature, threshold):
    return x[x[x.columns[feature]] <= threshold], x[x[x.columns[feature]] > threshold]

  # функция для подсчёта функционала качества
  def calculate_infromation_gain(self, origin, left, right):
    return (entropy(origin) - ((len(left) / len(origin)) * entropy(left) + (len(right) / len(origin)) * entropy(right)))

  # функция для выбора наилучшего разделения
  def tree_best_split(self, x, y):
    samples, features = x.shape
    max_information_gain = -1
    best_feature, best_threshold = -1, -1
    x_left, y_left, x_right, y_right = None, None, None, None
    for feature in range(features):
      for sample in range(samples):
        threshold = x.iloc[sample, feature]
        temp_x_left, temp_x_right = self.tree_split(x, feature, threshold)
        temp_y_left, temp_y_right = y[y.index.isin(temp_x_left.index)], y[y.index.isin(temp_x_right.index)]
        temp_information_gain = self.calculate_infromation_gain(y, temp_y_left, temp_y_right)
        if temp_information_gain > max_information_gain and len(temp_x_left) >= self.min_samples_leaf and len(temp_x_right) >= self.min_samples_leaf:
          best_feature, best_threshold = feature, threshold
          x_left, y_left, x_right, y_right = temp_x_left, temp_y_left, temp_x_right, temp_y_right
          max_information_gain = temp_information_gain
    return (best_feature, best_threshold, x_left, y_left, x_right, y_right)

  # функция условного роста дерева, пока есть возможности (глубина и другие критерии остановки соблюдены)
  def tree_grow(self, x, y, curr_depth=0):
    if (self.node_stop_check(y) or curr_depth == self.max_depth):
      return TreeNodeClass(y.value_counts().idxmax())
    else:
      feature, threshold, x_left, y_left, x_right, y_right = self.tree_best_split(x, y)
      LeftNode = self.tree_grow(x_left, y_left, curr_depth)
      RightNode = self.tree_grow(x_right, y_right, curr_depth)
      curr_depth += 1
      return TreeNodeDecision(feature, threshold, LeftNode, RightNode)
  
  # функция обучения
  def fit(self, x, y):
    return self.tree_grow(x, y)

  # функция вывода
  def __str__(self):
    return self.draw_tree(self.root)


In [7]:
# создание экземпляра класса с заданными параметрами
my_tree = MyDecisionTreeClassifier(min_samples_leaf=5, max_depth=5, purity=True)
# обучение
my_tree.root = my_tree.fit(X_train, y_train)
# вывод
print(my_tree)

print("\n")

# предсказание и оценка
y_predicted = my_tree.predict(X_test)
accuracy = np.round(accuracy_score(y_predicted, y_test), 2)
print(f"Accuracy: {accuracy}")


|--- feature_2 <= 1.9
|   |--- class: 0
|--- feature_2 > 1.9
|   |--- feature_3 <= 1.7
|   |   |--- feature_2 <= 4.7
|   |   |   |--- class: 1
|   |   |--- feature_2 > 4.7
|   |   |   |--- class: 1
|   |--- feature_3 > 1.7
|   |   |--- feature_0 <= 5.9
|   |   |   |--- class: 2
|   |   |--- feature_0 > 5.9
|   |   |   |--- class: 2


Accuracy: 0.93


### sklearn реализация

In [8]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree

# всё тоже самое, только из sklearn
clf = DecisionTreeClassifier(max_depth=5, min_samples_leaf=5)
clf.fit(X_train, y_train)

text_representation = tree.export_text(clf)
print(text_representation)

print("\n")

y_predicted = clf.predict(X_test)
accuracy = np.round(accuracy_score(y_predicted, y_test), 2)
print(f"Accuracy: {accuracy}")

|--- feature_2 <= 2.70
|   |--- class: 0
|--- feature_2 >  2.70
|   |--- feature_3 <= 1.75
|   |   |--- feature_2 <= 4.80
|   |   |   |--- class: 1
|   |   |--- feature_2 >  4.80
|   |   |   |--- class: 1
|   |--- feature_3 >  1.75
|   |   |--- feature_0 <= 6.00
|   |   |   |--- class: 2
|   |   |--- feature_0 >  6.00
|   |   |   |--- class: 2



Accuracy: 0.93


### Вывод
Реализация `sklearn` несколько отличается. Скорее всего, это связано с тем, как происходит поиск лучшего разбиения дерева — у меня он занимает `O(n ⋅ m)`, где `n` — количество объектов в выборке, а `m` — кол-во фич; в библиотеке, вероятно, этот момент как-то оптимизирован, потому что без оптимизации время работы на больших данных получилось бы слишком большим. В связи с этой оптимизацией и есть небольшие расхождения в порогах фич, иногда самих фичах, и точности.

##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
# id датасета на google drive с kaggle
file_id = "1DffMn70vE4vb3KX_B20QpifGPiXER-Ge"
!gdown $file_id

In [28]:
from sklearn.preprocessing import LabelEncoder

In [37]:
data = pd.read_csv('/content/churn.csv', index_col=0)
y = data["Exited"]

X = data

# создание One Hot Encoder для заданных категориальных признаков
dum_df = pd.get_dummies(X, columns=['Geography'], prefix=['geography'])
X = X.merge(dum_df)

# создание Label Encoder для заданных категориальных признаков
labelencoder = LabelEncoder()
X['Gender'] = labelencoder.fit_transform(X['Gender'])

# удаление ненужных признаков, старых категориальных и таргета
X = X.drop(columns=['CustomerId', 'Surname', 'Exited', 'Geography'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [31]:
# функция для создания бутстрап выборки
# принимает и отдает на вход два датафрейма, которые разбиваются одинаково, поэтому задается сид рандома
def get_bootstrap_sample(x, y, m):
  seed = int(np.random.rand() * np.power(10, 9))
  return x.sample(n=m,replace=True,random_state=seed), y.sample(n=m,replace=True,random_state=seed)

In [32]:
class RandomForest():
  def __init__(self, n_trees=100, max_depth=None, min_samples_leaf=1, criterion='gini'):
    self.n_trees = n_trees
    self.max_depth = max_depth
    self.min_samples_leaf = min_samples_leaf
    self.criterion = criterion
  
  # функция обучения, создает список деревьев-решений и важностей признаков
  def fit(self, X, y, m):
    self.trees = []
    self.features_importance = []
    for t in range(self.n_trees):
      tree = DecisionTreeClassifier(criterion=self.criterion, max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf,max_features='sqrt')
      tree_x, tree_y = get_bootstrap_sample(X, y, m)
      tree.fit(tree_x, tree_y)
      self.features_importance.append(tree.feature_importances_)
      self.trees.append(tree)

  # функция предсказания, проходится по каждому дереву, и присваивает самый частый предсказанный класс
  def predict(self, X):
    predictions = pd.DataFrame()
    for i, tree in enumerate(self.trees):
      predictions[i] = tree.predict(X)
    return np.round(predictions.mean(axis=1))

In [33]:
random_forest = RandomForest(n_trees=100, min_samples_leaf=1, criterion='gini')
random_forest.fit(X_train, y_train, m=1500)

y_predicted = random_forest.predict(X_test)
accuracy = np.round(accuracy_score(y_predicted, y_test), 4)

print(f"Accuracy: {accuracy}")

Accuracy: 0.8703


In [34]:
# расчет важности признаков — средняя важность каждого признака из всех деревьев 
features = pd.concat([pd.Series(X_test.columns, name="Feature"), pd.Series(np.mean(random_forest.features_importance, axis=0), name="Feature Importance")], axis=1)

features.sort_values(by=["Feature Importance"], ascending=False)

,Feature,Feature Importance
2,Age,0.233734
8,EstimatedSalary,0.140290
0,CreditScore,0.139170
4,Balance,0.135686
5,NumOfProducts,0.125831
3,Tenure,0.085876
7,IsActiveMember,0.043237
1,Gender,0.024629
10,geography_Germany,0.023229
6,HasCrCard,0.020275


### sklearn реализация

In [35]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=None, min_samples_leaf=1)

clf.fit(X_train, y_train)
fi = clf.feature_importances_

y_predicted = clf.predict(X_test)
accuracy = np.round(accuracy_score(y_predicted, y_test), 4)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8691


In [36]:
features["sklearn"] = clf.feature_importances_
features.sort_values(by=["sklearn"], ascending=False)

,Feature,Feature Importance,sklearn
2,Age,0.233734,0.236584
8,EstimatedSalary,0.140290,0.145826
0,CreditScore,0.139170,0.143073
4,Balance,0.135686,0.142617
5,NumOfProducts,0.125831,0.124633
3,Tenure,0.085876,0.084946
7,IsActiveMember,0.043237,0.041824
10,geography_Germany,0.023229,0.021811
1,Gender,0.024629,0.020038
6,HasCrCard,0.020275,0.018754
